# Nuclia 理解

>[Nuclia](https://nuclia.com) 可自动索引来自任何内部和外部源的非结构化数据，提供优化的搜索结果和生成式答案。它可以处理视频和音频转录、图像内容提取和文档解析。

`Nuclia 理解 API` 支持对非结构化数据进行处理，包括文本、网页、文档以及音频/视频内容。它会提取文本内容（必要时使用语音转文本或 OCR），识别实体，提取元数据、嵌入文件（如 PDF 中的图片）以及网页链接。它还可以提供内容摘要。

要使用 `Nuclia 理解 API`，您需要拥有一个 `Nuclia` 账户。您可以在 [https://nuclia.cloud](https://nuclia.cloud) 免费创建账户，然后[创建 NUA 密钥](https://docs.nuclia.dev/docs/docs/using/understanding/intro)。

In [ ]:
%pip install --upgrade --quiet  protobuf
%pip install --upgrade --quiet  nucliadb-protos

In [ ]:
import os

os.environ["NUCLIA_ZONE"] = "<YOUR_ZONE>"  # e.g. europe-1
os.environ["NUCLIA_NUA_KEY"] = "<YOUR_API_KEY>"

In [ ]:
from langchain_community.tools.nuclia import NucliaUnderstandingAPI

nua = NucliaUnderstandingAPI(enable_ml=False)

您可以使用 `push` 操作将文件推送到 Nuclia Understanding API。由于处理是异步进行的，结果可能与您推送文件的顺序不同。因此，您需要提供一个 `id` 来匹配结果和相应的文件。

In [ ]:
nua.run({"action": "push", "id": "1", "path": "./report.docx"})
nua.run({"action": "push", "id": "2", "path": "./interview.mp4"})

现在，你可以循环调用 `pull` 操作，直到获得 JSON 格式的结果。

In [ ]:
import time

pending = True
data = None
while pending:
    time.sleep(15)
    data = nua.run({"action": "pull", "id": "1", "path": None})
    if data:
        print(data)
        pending = False
    else:
        print("waiting...")

你也可以在 `async` 模式下一步完成，你只需要执行一个 push 操作，它会等待结果被 pull：

In [ ]:
import asyncio


async def process():
    data = await nua.arun(
        {"action": "push", "id": "1", "path": "./talk.mp4", "text": None}
    )
    print(data)


asyncio.run(process())

## 检索到的信息

Nuclia 返回以下信息：

- 文件元数据
- 提取的文本
- 嵌套文本（例如嵌入在图片中的文本）
- 摘要（仅当 `enable_ml` 设置为 `True` 时）
- 段落和句子拆分（通过其首尾字符的位置以及视频或音频文件的开始和结束时间定义）
- 命名实体：人物、日期、地点、组织等（仅当 `enable_ml` 设置为 `True` 时）
- 链接
- 缩略图
- 嵌入式文件
- 文本的向量表示（仅当 `enable_ml` 设置为 `True` 时）

注意：

  生成的文件（缩略图、提取的嵌入式文件等）将以 token 的形式提供。您可以使用 [`/processing/download` 端点](https://docs.nuclia.dev/docs/api#operation/Download_binary_file_processing_download_get) 下载它们。

  此外，在任何层级，如果某个属性的大小超过一定限制，它将被放入一个可下载文件中，并在文档中被文件指针替换。文件指针将由 `{"file": {"uri": "JWT_TOKEN"}}` 组成。规则是，如果消息大小超过 1000000 个字符，最大的部分将被移至可下载文件中。首先，压缩过程将以向量为目标。如果这还不够，它将以大型字段元数据为目标，最后将以提取的文本为目标。